<a href="https://colab.research.google.com/github/sovestinet27/MIPT_SQL/blob/main/%D0%90%D0%BA%D0%B8%D0%BD%D1%8C%D1%89%D0%B8%D0%BA%D0%BE%D0%B2_%D0%90%D1%80%D1%81%D0%B5%D0%BD%D0%B8%D0%B9_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B5%D0%B2%D0%B8%D1%87_%D0%94%D0%975.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install findspark

In [60]:
!pip install pyspark

In [61]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HotelData").getOrCreate()
df = spark.read.csv("Hotel.csv", header=True, inferSchema=True)

In [62]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- n_adults: integer (nullable = true)
 |-- n_children: integer (nullable = true)
 |-- weekend_nights: integer (nullable = true)
 |-- week_nights: integer (nullable = true)
 |-- meal_plan: string (nullable = true)
 |-- car_parking_space: integer (nullable = true)
 |-- room_type: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_room_price: double (nullable = true)
 |-- special_requests: integer (nullable = true)
 |-- status: string (nullable = true)



In [63]:
df.show(5, 0)

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|ID      |n_adults|n_children|weekend_nights|week_nights|meal_plan   |car_parking_space|room_type  |lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|status      |
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN00001|2       |0         |1             |2          |Meal Plan 1 |0                |Room_Type 1|224      |2017|10   |2   |Offline       |0             |0                     |0                             |65.0          |0      

In [64]:
from pyspark.sql.functions import col, avg

filtered_df = df.filter(col("status") == "Not_Canceled")
filtered_df = filtered_df.withColumn("total_nights", col("weekend_nights") + col("week_nights"))
result_df = filtered_df.groupBy("year", "month")\
                             .agg(avg("total_nights").alias("average_total_nights"))
result_df = result_df.orderBy("year", "month")
result_df.show(result_df.count(), False)

+----+-----+--------------------+
|year|month|average_total_nights|
+----+-----+--------------------+
|2017|7    |3.0166666666666666  |
|2017|8    |2.7189384800965017  |
|2017|9    |2.6550783912747105  |
|2017|10   |2.7032898820608318  |
|2017|11   |2.7241935483870967  |
|2017|12   |3.043046357615894   |
|2018|1    |2.7414141414141415  |
|2018|2    |2.6891679748822606  |
|2018|3    |3.0392038600723765  |
|2018|4    |2.924755887421022   |
|2018|5    |2.8054545454545456  |
|2018|6    |2.596757322175732   |
|2018|7    |3.1938088829071334  |
|2018|8    |3.1544117647058822  |
|2018|9    |2.786425902864259   |
|2018|10   |2.8910186199342824  |
|2018|11   |2.9784511784511785  |
|2018|12   |3.2521891418563924  |
+----+-----+--------------------+



In [65]:
from pyspark.sql.functions import sequence, to_date, lit, explode

start_date = to_date(lit("2017-01-01"))
end_date = to_date(lit("2018-12-31"))
calendar_df = spark.range(1).select(explode(sequence(start_date, end_date)).alias("calendar_dt"))
calendar_df.printSchema()
calendar_df.show(10)

root
 |-- calendar_dt: date (nullable = false)

+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
| 2017-01-06|
| 2017-01-07|
| 2017-01-08|
| 2017-01-09|
| 2017-01-10|
+-----------+
only showing top 10 rows


In [74]:
filtered_df = df.filter(col("status") == "Not_Canceled")
filtered_df = filtered_df.withColumn("total_nights", col("weekend_nights") + col("week_nights"))
result_df = filtered_df.groupBy("year", "month")\
                             .agg(avg("total_nights").alias("average_total_nights"))

result_df = result_df.orderBy("year", "month")
result_df.show(result_df.count(), False)

+----+-----+--------------------+
|year|month|average_total_nights|
+----+-----+--------------------+
|2017|7    |3.0166666666666666  |
|2017|8    |2.7189384800965017  |
|2017|9    |2.6550783912747105  |
|2017|10   |2.7032898820608318  |
|2017|11   |2.7241935483870967  |
|2017|12   |3.043046357615894   |
|2018|1    |2.7414141414141415  |
|2018|2    |2.6891679748822606  |
|2018|3    |3.0392038600723765  |
|2018|4    |2.924755887421022   |
|2018|5    |2.8054545454545456  |
|2018|6    |2.596757322175732   |
|2018|7    |3.1938088829071334  |
|2018|8    |3.1544117647058822  |
|2018|9    |2.786425902864259   |
|2018|10   |2.8910186199342824  |
|2018|11   |2.9784511784511785  |
|2018|12   |3.2521891418563924  |
+----+-----+--------------------+



In [75]:
from pyspark.sql.functions import count

df_2018 = df.filter(col("year") == 2018)
total_bookings_per_month = df_2018.groupBy("month")\
                                    .agg(count("ID").alias("total_bookings"))
canceled_bookings_per_month = df_2018.filter(col("status") == "Canceled")\
                                    .groupBy("month")\
                                    .agg(count("ID").alias("canceled_bookings"))
cancellation_rates = total_bookings_per_month.join(canceled_bookings_per_month, "month", "left_outer")\
                                                .withColumn("cancellation_percentage",
                                                            (col("canceled_bookings") / col("total_bookings")) * 100)
cancellation_rates = cancellation_rates.na.fill(0, subset=["canceled_bookings"])
top_3_months_2018 = cancellation_rates.orderBy(col("cancellation_percentage").desc())\
                                        .limit(3)
top_3_months_2018.show()

+-----+--------------+-----------------+-----------------------+
|month|total_bookings|canceled_bookings|cancellation_percentage|
+-----+--------------+-----------------+-----------------------+
|    8|          2799|             1303|      46.55234012147195|
|   10|          3404|             1578|       46.3572267920094|
|    9|          2962|             1356|      45.77987846049966|
+-----+--------------+-----------------+-----------------------+



In [76]:
confirmed_bookings_df = df.filter(col("status") == "Not_Canceled")
average_lead_time_per_month = confirmed_bookings_df.groupBy("year", "month")\
                                                        .agg(avg("lead_time").alias("average_lead_time"))
average_lead_time_per_month = average_lead_time_per_month.orderBy("year", "month")
average_lead_time_per_month.show(average_lead_time_per_month.count(), False)

+----+-----+------------------+
|year|month|average_lead_time |
+----+-----+------------------+
|2017|7    |130.73333333333332|
|2017|8    |35.08082026537998 |
|2017|9    |51.72188139059305 |
|2017|10   |55.885164494103044|
|2017|11   |33.28225806451613 |
|2017|12   |46.746136865342166|
|2018|1    |34.87171717171717 |
|2018|2    |30.53375196232339 |
|2018|3    |43.19420989143546 |
|2018|4    |62.492820218265365|
|2018|5    |60.98909090909091 |
|2018|6    |70.63546025104603 |
|2018|7    |86.88021534320323 |
|2018|8    |83.08622994652407 |
|2018|9    |63.32316313823163 |
|2018|10   |73.24370208105148 |
|2018|11   |44.25387205387205 |
|2018|12   |69.75306479859896 |
+----+-----+------------------+



In [77]:
confirmed_bookings_df = df.filter(col("status") == "Not_Canceled")
pivot_df = confirmed_bookings_df.groupBy("year", "month") \
                                 .pivot("room_type") \
                                 .agg(avg("avg_room_price").alias("average_revenue"))
pivot_df = pivot_df.orderBy("year", "month")
pivot_df.show(pivot_df.count(), False)

+----+-----+------------------+------------------+-----------+------------------+------------------+------------------+------------------+
|year|month|Room_Type 1       |Room_Type 2       |Room_Type 3|Room_Type 4       |Room_Type 5       |Room_Type 6       |Room_Type 7       |
+----+-----+------------------+------------------+-----------+------------------+------------------+------------------+------------------+
|2017|7    |77.95102564102567 |98.2              |NULL       |NULL              |0.0               |NULL              |NULL              |
|2017|8    |89.63858666666663 |42.93888888888889 |NULL       |71.4090909090909  |0.0               |149.07065217391303|NULL              |
|2017|9    |99.672028127313   |54.96333333333333 |NULL       |108.70716049382712|41.775            |178.2426923076923 |159.75            |
|2017|10   |90.10942622950822 |82.24133333333334 |65.0       |103.52048309178743|0.0               |172.9085714285714 |0.0               |
|2017|11   |72.694376367614

In [78]:
from pyspark.sql.functions import sum

repeated_guest_confirmed_bookings = df.filter(
    (col("status") == "Not_Canceled") &
    (col("repeated_guest") == 1)
)
total_revenue_repeated_guests = repeated_guest_confirmed_bookings\
                                .agg(sum("avg_room_price").alias("total_revenue"))\
                                .collect()[0]["total_revenue"]
top_5_repeated_guest_bookings = repeated_guest_confirmed_bookings\
                                    .orderBy(col("avg_room_price").desc())\
                                    .limit(5)
top_5_repeated_guest_bookings.show(truncate=False)

+--------+--------+----------+--------------+-----------+-----------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|ID      |n_adults|n_children|weekend_nights|week_nights|meal_plan  |car_parking_space|room_type  |lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|status      |
+--------+--------+----------+--------------+-----------+-----------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN09923|2       |1         |1             |2          |Meal Plan 1|1                |Room_Type 7|8        |2018|12   |30  |Corporate     |1             |0                     |1                             |220.0         |2          

In [71]:
from pyspark.sql.functions import to_date, concat_ws, date_add, lit, coalesce, try_to_timestamp
from pyspark.sql.types import IntegerType

hotel_capacity = 400
confirmed_bookings_with_dates = df.filter(col("status") == "Not_Canceled") \
    .withColumn("check_in_date_str", concat_ws("-", col("year"), col("month"), col("date"))) \
    .withColumn("check_in_date", try_to_timestamp(col("check_in_date_str"), lit("yyyy-M-d")).cast("date")) \
    .filter(col("check_in_date").isNotNull()) \
    .withColumn("check_out_date", date_add(col("check_in_date"), col("weekend_nights") + col("week_nights"))) \
    .select("ID", "n_adults", "n_children", "check_in_date", "check_out_date")
joined_df = calendar_df.alias("c").join(
    confirmed_bookings_with_dates.alias("h"),
    (col("c.calendar_dt") >= col("h.check_in_date")) & (col("c.calendar_dt") < col("h.check_out_date")),
    "left_outer"
)
daily_occupancy_df = joined_df.groupBy(col("c.calendar_dt")) \
    .agg(
        sum(col("h.n_adults") + col("h.n_children")).alias("total_guests_for_day")
    ) \
    .withColumn("total_guests_for_day", coalesce(col("total_guests_for_day"), lit(0)).cast(IntegerType()))
daily_occupancy_df = daily_occupancy_df.withColumn(
    "occupancy_percentage",
    (col("total_guests_for_day") / lit(hotel_capacity)) * 100
)
daily_occupancy_df = daily_occupancy_df.orderBy(col("c.calendar_dt").desc())
daily_occupancy_df.show(daily_occupancy_df.count(), False)

+-----------+--------------------+--------------------+
|calendar_dt|total_guests_for_day|occupancy_percentage|
+-----------+--------------------+--------------------+
|2018-12-31 |562                 |140.5               |
|2018-12-30 |572                 |143.0               |
|2018-12-29 |542                 |135.5               |
|2018-12-28 |507                 |126.75              |
|2018-12-27 |552                 |138.0               |
|2018-12-26 |422                 |105.5               |
|2018-12-25 |397                 |99.25               |
|2018-12-24 |373                 |93.25               |
|2018-12-23 |341                 |85.25               |
|2018-12-22 |282                 |70.5                |
|2018-12-21 |247                 |61.75000000000001   |
|2018-12-20 |240                 |60.0                |
|2018-12-19 |228                 |56.99999999999999   |
|2018-12-18 |258                 |64.5                |
|2018-12-17 |274                 |68.5          

In [72]:
from pyspark.sql.functions import expr

hotel_capacity = 400
confirmed_bookings_with_dates = df.filter(col("status") == "Not_Canceled") \
    .withColumn("check_in_date_str", concat_ws("-", col("year"), col("month"), col("date"))) \
    .withColumn("check_in_date", try_to_timestamp(col("check_in_date_str"), lit("yyyy-M-d")).cast("date")) \
    .filter(col("check_in_date").isNotNull()) \
    .withColumn("check_out_date", expr("date_add(check_in_date, weekend_nights + week_nights)")) \
    .select("ID", "n_adults", "n_children", "check_in_date", "check_out_date")
joined_df = calendar_df.alias("c").join(
    confirmed_bookings_with_dates.alias("h"),
    (col("c.calendar_dt") >= col("h.check_in_date")) & (col("c.calendar_dt") < col("h.check_out_date")),
    "left_outer"
)
daily_occupancy_df = joined_df.groupBy(col("c.calendar_dt")) \
    .agg(
        sum(col("h.n_adults") + col("h.n_children")).alias("total_guests_for_day")
    ) \
    .withColumn("total_guests_for_day", coalesce(col("total_guests_for_day"), lit(0)).cast(IntegerType()))
daily_occupancy_df = daily_occupancy_df.withColumn(
    "occupancy_percentage",
    (col("total_guests_for_day") / lit(hotel_capacity)) * 100
)
daily_occupancy_df = daily_occupancy_df.orderBy(col("c.calendar_dt").desc())
daily_occupancy_df.show(daily_occupancy_df.count(), False)

+-----------+--------------------+--------------------+
|calendar_dt|total_guests_for_day|occupancy_percentage|
+-----------+--------------------+--------------------+
|2018-12-31 |562                 |140.5               |
|2018-12-30 |572                 |143.0               |
|2018-12-29 |542                 |135.5               |
|2018-12-28 |507                 |126.75              |
|2018-12-27 |552                 |138.0               |
|2018-12-26 |422                 |105.5               |
|2018-12-25 |397                 |99.25               |
|2018-12-24 |373                 |93.25               |
|2018-12-23 |341                 |85.25               |
|2018-12-22 |282                 |70.5                |
|2018-12-21 |247                 |61.75000000000001   |
|2018-12-20 |240                 |60.0                |
|2018-12-19 |228                 |56.99999999999999   |
|2018-12-18 |258                 |64.5                |
|2018-12-17 |274                 |68.5          